In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/26 11:07:02 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/04/26 11:07:02 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/04/26 11:07:02 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/04/26 11:07:02 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
directory = "/home/ithingvv34/data-engineering/spark/data"
filename = "fhvhv_tripdata_2020-03.csv"

In [4]:
data = spark.read.csv(f"file:///{directory}/{filename}", inferSchema = True, header = True)

In [5]:
data.show(5)

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [6]:
data.createOrReplaceTempView("mobility_data")

In [7]:
spark.sql("select * from mobility_data limit 5").show()

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|_c0|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [8]:
spark.sql("select pickup_date, count(*) as trips from (select split(pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date ").show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-26|141607|
| 2020-03-25|141089|
| 2020-03-02|648990|
| 2020-03-03|697880|
| 2020-03-05|731165|
| 2020-03-06|872012|
| 2020-03-16|391518|
| 2020-03-20|261900|
| 2020-03-24|141687|
| 2020-03-04|707879|
| 2020-03-10|626474|
| 2020-03-11|628601|
| 2020-03-12|643257|
| 2020-03-01|784260|
| 2020-03-09|628940|
| 2020-03-13|660914|
| 2020-03-22|162165|
| 2020-03-19|252773|
| 2020-03-27|159339|
| 2020-03-28|138456|
+-----------+------+
only showing top 20 rows



In [9]:
spark.sql("select pickup_date, count(*) as trips from (select split(pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date ").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#361]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#360]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#360], [pickup_date#360, count(1) AS trips#361L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(pickup_datetime#22,  , -1)[0] AS pickup_date#360]
      +- SubqueryAlias mobility_data
         +- Relation[_c0#16,hvfhs_license_num#17,dispatching_base_num#18,originating_base_num#19,request_datetime#20,on_scene_datetime#21,pickup_datetime#22,dropoff_datetime#23,PULocationID#24,DOLocationID#25,trip_miles#26,trip_time#27,base_passenger_fare#28,tolls#29,bcf#30,sales_tax#31,congestion_surcharge#32,airport_fee#33,tips#34,driver_pay#35,shared_request_flag#36,shared_match_flag#37,access_a_ride_flag#38,wav_request_flag#39,wav_match_